In [ ]:
import pandas as pd 
import os 
import warnings
import networkx as nx 

warnings.filterwarnings('ignore')

In [ ]:
pwd = os.getcwd()

In [ ]:
# convert data to trans2vec format
data = {}
label = {}

for filename in os.listdir(pwd + '/../original_data/钓鱼一阶节点/'):
    if filename.endswith('.csv'):
        df = pd.read_csv(pwd + '/../original_data/钓鱼一阶节点/' + filename)
        G = nx.from_pandas_edgelist(df, source='From', target='To', edge_attr=('Value', 'TimeStamp'))
        addr = filename.split('.')[0]
        data[addr] = G
        label[addr] = 0

for filename in os.listdir(pwd + '/../original_data/非钓鱼一阶节点/'):
    if filename.endswith('.csv'):
        df = pd.read_csv(pwd + '/../original_data/非钓鱼一阶节点/' + filename)
        G = nx.from_pandas_edgelist(df, source='From', target='To', edge_attr=('Value', 'TimeStamp'))
        addr = filename.split('.')[0]
        data[addr] = G
        label[addr] = 1